## Live Demo

In [ ]:
# %pip install tensorflow
# %pip install Pillow
# %pip install matplotlib
# %pip install opencv-python
# %pip install vonage
# %pip install python-dotenv

# %pip install tensorflow
# %pip install Pillow
# %pip install matplotlib
# %pip install opencv-python
# %pip install vonage
# %pip install python-dotenv

import sys
import os
import warnings
# import logging

# logging.basicConfig(level=logging.DEBUG)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.filterwarnings('ignore')  # <-- Comment out or remove this line

class DevNull:
    def write(self, msg): pass
    def flush(self): pass

sys.stderr = DevNull()      # Ignore Python warnings (remove or comment out)

# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'  # Show all TensorFlow logs
# warnings.filterwarnings('default')        # Show all warnings

# # Restore sys.stderr to default if it was redirected
# if hasattr(sys, '__stderr__'):
#     sys.stderr = sys.__stderr__

import cv2
from PIL import Image
from fall_prediction import Fall_prediction
import matplotlib.pyplot as plt
import requests
import datetime
from dotenv import load_dotenv
import os
import threading
import queue
import time

frame_queue = queue.Queue(maxsize=100)  # Limit queue size to avoid memory issues

def capture_camera():
    print("Starting camera capture...")
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        record_time = datetime.datetime.now()
        # Convert OpenCV BGR image to PIL Image
        pil_img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        frame_queue.put((pil_img, record_time))
        time.sleep(0.5)  # Sleep for 0.5 seconds to control capture rate
    cap.release()
    frame_queue.put(None)  # Sentinel to signal end

def handle_frame():
    frame_buffer = []
    while True:
        print("Waiting for frames...")
        item = frame_queue.get()
        if item is None:
            break  # Exit loop if sentinel received 
        pil_img, record_time = item
        
        frame_buffer.append({
            'record_time': record_time,
            'img': pil_img
        })
    
        # Keep only the last 3 frames
        if len(frame_buffer) > 3:
            frame_buffer.pop(0)

        # Run prediction when we have 3 frames
        if len(frame_buffer) == 3:
            fig = plt.figure(figsize=(12,12))
            for idx, item in enumerate(frame_buffer):
                no = idx+1
                fig.add_subplot(1, 3, no)
                plt.imshow(item['img'])
                plt.title(item['record_time'], fontsize=8)
                plt.axis("off")
            plt.show()

            img_list = [item['img'] for item in frame_buffer]
            response = Fall_prediction(*img_list)
            if response:
                print("There is", response['category'])
                print("Confidence :", response['confidence'])
                print("Angle :", response['angle'])
                print("Keypoint_corr :", response['keypoint_corr'])

                # Call /record_fall API
                try:
                    api_response = requests.post(
                        f"{os.getenv('SIGNAL_SERVER_URL')}/record_fall",
                        json={
                            "source": "camera",
                            "patient_id": 1,
                            "detected_at": datetime.datetime.now().isoformat()
                        }
                    )
                    print("API response:", api_response.json())
                except Exception as e:
                    print("Failed to call /record_fall:", e)
            else:
                print("There is no fall detection...")

# Start threads
capture_thread = threading.Thread(target=capture_camera)
handle_thread = threading.Thread(target=handle_frame)

capture_thread.start()
handle_thread.start()

capture_thread.join()
handle_thread.join()